<a href="https://colab.research.google.com/github/keenanpepper/self-ablating-transformers/blob/main/Self_ablating_transformer_with_smooth_top_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/keenanpepper/self-ablating-transformers

Cloning into 'self-ablating-transformers'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 74 (delta 7), reused 14 (delta 4), pack-reused 42 (from 1)
Receiving objects: 100% (74/74), 57.09 MiB | 12.79 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [3]:
!git config --global user.email "keenanpepper@gmail.com"
!git config --global user.name "Keenan Pepper"

In [2]:
cd self-ablating-transformers/

/content/self-ablating-transformers


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.8/174.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datasets
from tqdm.notebook import tqdm
import numpy as np

In [4]:
from model.gpt_neo import GPTNeoWithSelfAblation, soft_top_k
from model.config import GPTNeoWithSelfAblationConfig

In [5]:
model = GPTNeoWithSelfAblation(GPTNeoWithSelfAblationConfig(hidden_size=128))

In [6]:
model

GPTNeoWithSelfAblation(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 128)
    (wpe): Embedding(2048, 128)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlockWithSelfAblation(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionWithSelfAblation(
          (attention): ModuleDict(
            (k_proj): Linear(in_features=128, out_features=128, bias=False)
            (v_proj): Linear(in_features=128, out_features=128, bias=False)
            (q_proj): Linear(in_features=128, out_features=128, bias=False)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLPWithSelfAblation(
          (c_fc): Linear(in_features=128, out_features=512, bias=True)
          (c_proj): Linear(in_features=512, out_features=128, bias=True)
          (act): NewGELUActivation()
        )
      )
    )
    (ln_f): LayerNorm((128,),

In [7]:
model(torch.zeros((1,1), dtype=torch.long))

{'logits': tensor([[[ 0.0269, -0.3266,  0.8674,  ..., -1.3128, -0.1785,  0.3069]]],
        grad_fn=<UnsafeViewBackward0>),
 'logits_ablated': tensor([[[ 0.4077,  0.1292,  0.4486,  ..., -0.9105,  0.3658,  0.1320]]],
        grad_fn=<UnsafeViewBackward0>),
 'L_base': None,
 'L_ablated': None,
 'loss': None,
 'attention_ablations': tensor([[[[0.0218, 0.0235, 0.0227,  ..., 0.1646, 0.0247, 0.0432],
           [0.0115, 0.0421, 0.0317,  ..., 0.0584, 0.0607, 0.0813],
           [0.0195, 0.0309, 0.0274,  ..., 0.0481, 0.0359, 0.1408],
           ...,
           [0.1007, 0.0483, 0.1678,  ..., 0.0824, 0.0918, 0.0668],
           [0.0808, 0.0720, 0.0440,  ..., 0.0658, 0.0338, 0.0466],
           [0.0242, 0.0059, 0.0804,  ..., 0.0087, 0.0354, 0.0556]]]],
        grad_fn=<ViewBackward0>),
 'neuron_ablations': tensor([[[[0.0346, 0.0414, 0.0183,  ..., 0.0068, 0.0105, 0.0047],
           [0.0394, 0.0049, 0.0138,  ..., 0.0498, 0.0156, 0.0215],
           [0.0260, 0.0056, 0.0282,  ..., 0.0063, 0.0049, 0.

In [8]:
ds = datasets.load_dataset("tdooms/TinyStories")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
from utils import prepare_data

In [10]:
data, tokenizer = prepare_data(separator_token="<|endoftext|>")
print(f"Processed data shape: {data.shape}")
print(f"First 10 tokens: {data[:10]}")
print(f"Last 10 tokens: {data[-10:]}")

# Decode a small portion to verify
sample = data[:10000]
decoded = tokenizer.decode(sample.tolist())
print("\nSample decoded text:")
print(decoded[:5000] + "...")  # Print first 500 characters

train.bin already exists. Skipping processing.
Processed data shape: (396967656,)
First 10 tokens: [ 8888    11 19919   373   845  6568    13   679   373  1016]
Last 10 tokens: [  262 25103 12023   290  8359   257 12625  8073    13 50256]

Sample decoded text:
Today, Tommy was very excited. He was going flying with his mom and dad to a special place. There was a big green flag waving in the wind when they arrived. Tommy was so happy to see it. 

He hopped out of the car and ran up to the flag. He couldn't believe how big it was! He wanted to reach out and touch it. His mom said he could and Tommy smiled.

He waved his little arms, trying to make the flag move. But the wind was too strong. His dad said, "Let me help you, Tommy." He reached out and grabbed the green flag and waved it back and forth. 

Now it was Tommy's turn! He clapped his hands and laughed as he tried to move the flag. He waved it for a long time until he was too tired to do it anymore. The big green flag was so exciti

In [11]:
prepare_data(split="validation", output_file="validation.bin")

validation.bin already exists. Skipping processing.


(memmap([ 7454,  2402,   257, ..., 20567,    13, 50256], dtype=uint16),
 <Encoding 'gpt2'>)

In [30]:
class TrainingConfig:
    train_file = "train.bin"
    val_file = "validation.bin"
    block_size = 256
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_batches = 120000
    batch_size = 64
    learning_rate = 4e-3
    weight_decay = 0.0
    max_grad_norm = 1.0
    save_path = "best_model.pt"
    eval_iters = 100
    log_interval = 1000

In [31]:
class BatchGenerator:
    def __init__(self, data_file, block_size, batch_size, device):
        self.data_file = data_file
        self.block_size = block_size
        self.batch_size = batch_size
        self.device = device
        self.device_type = 'cuda' if 'cuda' in device.type else 'cpu'

    def get_batch(self, shifted=True):
        # We recreate np.memmap every batch to avoid a memory leak, as per
        # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
        data = np.memmap(self.data_file, dtype=np.uint16, mode='r')

        # Generate random starting indices
        ix = torch.randint(len(data) - self.block_size, (self.batch_size,))

        shift = 1 if shifted else 0
        # Create input and target tensors
        x = torch.stack([torch.from_numpy((data[i:i+self.block_size]).astype(np.int64)) for i in ix])
        y = torch.stack([torch.from_numpy((data[i+shift:i+shift+self.block_size]).astype(np.int64)) for i in ix])

        if self.device_type == 'cuda':
            # Pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
            x, y = x.pin_memory().to(self.device, non_blocking=True), y.pin_memory().to(self.device, non_blocking=True)
        else:
            x, y = x.to(self.device), y.to(self.device)

        return x, y

In [32]:
class LossEstimator:
    def __init__(self, model, train_batch_gen, val_batch_gen, eval_iters):
        self.model = model
        self.train_batch_gen = train_batch_gen
        self.val_batch_gen = val_batch_gen
        self.eval_iters = eval_iters

    def estimate_loss(self):
        stat_names = ["loss", "L_base", "L_ablated", "attention_ablation_mask_density", "neuron_ablation_mask_density"]
        out = {}
        for stat in stat_names:
            out[stat] = {}
        self.model.eval()
        with torch.inference_mode():
            for split, batch_gen in [('train', self.train_batch_gen), ('val', self.val_batch_gen)]:
                stats = {}
                for stat in stat_names:
                    stats[stat] = torch.zeros(self.eval_iters)
                for k in tqdm(range(self.eval_iters)):
                    X, Y = batch_gen.get_batch()
                    ret = self.model(X, Y)
                    for stat in stat_names:
                        stats[stat][k] = ret[stat].item()
                for stat in stat_names:
                    out[stat][split] = stats[stat].mean()
        self.model.train()
        return out

    def estimate_loss_pretrainedformat(self):
        out = {}
        self.model.eval()
        with torch.inference_mode():
            for split, batch_gen in [('train', self.train_batch_gen), ('val', self.val_batch_gen)]:
                losses = torch.zeros(self.eval_iters)
                for k in tqdm(range(self.eval_iters)):
                    X, Y = batch_gen.get_batch(shifted=False)
                    ret = self.model(X, labels=Y)
                    losses[k] = ret["loss"].item()
                out[split] = losses.mean()
        self.model.train()
        return out

In [33]:
def train_gptneo(model, config):
    train_batch_gen = BatchGenerator(config.train_file, config.block_size, config.batch_size, config.device)
    val_batch_gen = BatchGenerator(config.val_file, config.block_size, config.batch_size, config.device)
    loss_estimator = LossEstimator(model, train_batch_gen, val_batch_gen, config.eval_iters)

    model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.num_batches)

    best_val_loss = float('inf')

    for iteration in tqdm(range(config.num_batches)):
        model.train()

        # Get batch
        x, y = train_batch_gen.get_batch()

        # Forward pass
        loss = model(x, targets=y)["loss"]

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        if config.max_grad_norm:
            nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)

        optimizer.step()
        scheduler.step()

        # Logging
        if (iteration + 1) % config.log_interval == 0:
            stats = loss_estimator.estimate_loss()
            print(f"Iteration {iteration}: train loss {stats['loss']['train']:.4f}, val loss {stats['loss']['val']:.4f}")
            print(f"train L_base {stats['L_base']['train']:.4f}, val L_base {stats['L_base']['val']:.4f}")
            print(f"train L_ablated {stats['L_ablated']['train']:.4f}, val L_ablated {stats['L_ablated']['val']:.4f}")
            print(f"train attention ablation mask density {stats['attention_ablation_mask_density']['train']:.4f}, val {stats['attention_ablation_mask_density']['val']:.4f}")
            print(f"train neuron ablation mask density {stats['neuron_ablation_mask_density']['train']:.4f}, val {stats['neuron_ablation_mask_density']['val']:.4f}")
            print(f"The current learning rate: {optimizer.param_groups[0]['lr']:.4f}")

            # Save best model
            if stats['loss']['val'] < best_val_loss:
                best_val_loss = stats['loss']['val']
                torch.save(model.state_dict(), config.save_path)
                print(f"New best model saved to {config.save_path}")

    print("Training completed!")

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
config = GPTNeoWithSelfAblationConfig(hidden_size=128, max_position_embeddings=256)
model = GPTNeoWithSelfAblation(config)

training_config = TrainingConfig()

train_gptneo(model, training_config)

  0%|          | 0/120000 [00:00<?, ?it/s]